In [ ]:
import heapq
import numpy as np
import open3d as o3d
from scipy.spatial.distance import cdist
from scipy.spatial.transform import Rotation as R

In [4]:
def compute_3d_correspondences(source_points: list, target_point: tuple, depth: list, K: list, max_distance = 3):
    source_points_3d = []
    target_points_3d = []

    # Lift first source point and respective target point
    source_points_3d.append(project_2d_to_3d(source_points[0], K[0], depth[0][source_points[0][::-1]]))
    target_points_3d.append(project_2d_to_3d(target_point, K[1], depth[1][target_point[::-1]]))

    # For the other source points sample points around the target point
    sampled_points, already_sampled_points = [], set()
    
    while len(sampled_points) < len(source_points) - 1:
        # Random distance from center (uniformly between 0 and max_distance)
        distance = np.random.uniform(0, max_distance)
        
        # Random angle in radians
        angle = np.random.uniform(0, 2 * np.pi)
        
        # Convert polar coordinates (distance, angle) to Cartesian coordinates
        x = target_point[0] + distance * np.cos(angle)
        y = target_point[1] + distance * np.sin(angle)

        if (x, y) not in already_sampled_points:
            sampled_points.append((int(x), int(y)))
            already_sampled_points.add((int(x), int(y)))

    # Lift remaining source points and sampled points to 3D
    for x in range(len(sampled_points)):
        source_points_3d.append(project_2d_to_3d(source_points[1+x], K[0], depth[0][source_points[1+x][::-1]]))
        target_points_3d.append(project_2d_to_3d(sampled_points[x], K[1], depth[1][sampled_points[x][::-1]]))

    return np.array(source_points_3d), np.array(target_points_3d)

In [43]:
def compute_3d_correspondences2(source_points: list, target_points: tuple, depth: list, K: list, max_distance = 3):
    source_points_3d = []
    target_points_3d = []

    for point in source_points:
        source_points_3d.append(project_2d_to_3d(point, K[0], depth[0][point[::-1]]))

    for point in target_points:
        target_points_3d.append(project_2d_to_3d(point, K[1], depth[1][point[::-1]]))

    return np.array(source_points_3d), np.array(target_points_3d)

In [44]:
def map_point_to_original(resized_point, original_size, resized_size):
    """
    Map a point from the resized image to the original image coordinates.

    Args:
    - resized_point: Tuple (x, y) representing the coordinates in the resized image.
    - original_size: Tuple (original_width, original_height) for the original image size.
    - resized_size: Tuple (resized_width, resized_height) for the resized image size.

    Returns:
    - original_point: Tuple (x, y) representing the mapped coordinates in the original image.
    """
    
    # Extract coordinates
    x_resized, y_resized = resized_point
    original_width, original_height = original_size
    resized_width, resized_height = resized_size
    
    # Compute the scaling factors
    scale_x = original_width / resized_width
    scale_y = original_height / resized_height
    
    # Map the point in the resized image to the original image
    x_original = x_resized * scale_x
    y_original = y_resized * scale_y
    
    return (int(x_original), int(y_original))

In [45]:
def upsample_2d_point(point: tuple, original_size: tuple, resized_size: tuple):
    return map_point_to_original(point, original_size, resized_size)

In [ ]:
def downsample_2d_point(original_point, original_size, resized_size):
    """
    Compute the corresponding point in a resized image.
    
    Args:
        original_point (tuple): (x, y) coordinates of the point in the original image.
        original_size (tuple): (width, height) of the original image.
        resized_size (tuple): (width, height) of the resized image.
    
    Returns:
        tuple: (x, y) coordinates of the point in the resized image.
    """
    x, y = original_point
    orig_width, orig_height = original_size
    resized_width, resized_height = resized_size
    
    # Scale the point
    x_new = x * resized_width / orig_width
    y_new = y * resized_height / orig_height
    
    return (int(x_new), int(y_new))

In [ ]:
from scipy.spatial import cKDTree
import numpy as np

def map_points_to_nearest_pointcloud(pointcloud, points):
    """
    Maps 3D points to their nearest neighbor in a given point cloud.

    Args:
        pointcloud (np.ndarray): Point cloud as an (N, 3) array of 3D points.
        points (np.ndarray): Set of points to map as an (M, 3) array of 3D points.

    Returns:
        np.ndarray: Array of the nearest point cloud points for each input point.
    """
    pointcloud_np = pointcloud
    
    # Build a KDTree from the point cloud
    tree = cKDTree(pointcloud_np)
    
    # Query the tree for distances and indices of the nearest neighbors
    _, indices = tree.query(points)
    
    # Map input points to their nearest point in the point cloud
    nearest_points = pointcloud_np[indices]
    
    return nearest_points

In [46]:
from PIL import Image, ImageDraw

def sanity_check(point: tuple, img):
    image = copy.deepcopy(img)
    
    draw = ImageDraw.Draw(image)
    
    point = point
    radius = 5
    color = (255, 0, 0)
    
    draw.ellipse(
        [point[0] - radius, point[1] - radius, point[0] + radius, point[1] + radius],
        fill=color
    )
    
    image.save("sanity_check.jpg")

In [6]:
def compute_3d_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

In [48]:
def compute_matches(pc1_np, pc2_np, R, t, threshold=0.02):
    """
    Count the number of matching points after applying a transformation to pc1.
    
    Args:
        pc1 (o3d.geometry.PointCloud): First point cloud (source).
        pc2 (o3d.geometry.PointCloud): Second point cloud (target).
        R (np.ndarray): Rotation matrix (3x3).
        t (np.ndarray): Translation vector (3,).
        threshold (float): Distance threshold for matching points.
    
    Returns:
        int: Number of matching points.
    """
    # Apply the transformation
    transformed_pc1_np = (R @ pc1_np.T).T + t  # Transform pc1 (broadcast t over rows)
    
    # Build a KDTree for efficient nearest-neighbor search
    tree = cKDTree(pc2_np)
    
    # Query the KDTree for each point in the transformed point cloud
    distances, indices = tree.query(transformed_pc1_np, k=1)
    
    # Count matches within the distance threshold
    num_matches = np.sum(distances < threshold)

    return num_matches

In [3]:
def center_point_cloud(pc):
    """
    Centers an Open3D PointCloud by translating its centroid to the origin.
    
    Parameters:
    - pc (o3d.geometry.PointCloud): The input point cloud to be centered.
    
    Returns:
    - o3d.geometry.PointCloud: A new point cloud centered at the origin.
    """
    # Convert point cloud to numpy array
    points = np.asarray(copy.deepcopy(pc.points))

    # Compute centroid
    centroid = np.mean(points, axis=0)
    
    # # Subtract the centroid to center the point cloud
    # centered_points = points - centroid
    
    # # Create a new Open3D PointCloud object
    # centered_pc = copy.deepcopy(pc)
    # centered_pc.points = o3d.utility.Vector3dVector(centered_points)
    
    # return centered_pc, centroid
    return pc, centroid

In [4]:
def project_3d_to_2d(point_3d, K):
    """
    Project a 3D point to 2D using the intrinsic matrix.
    
    :param point_3d: A numpy array or list of [x, y, z] representing the 3D point.
    :param K: Intrinsic matrix of the camera.
    :return: A tuple (u, v) representing the 2D image coordinates.
    """
    # Convert the 3D point into homogeneous coordinates
    point_3d_homogeneous = np.array([point_3d[0], point_3d[1], point_3d[2], 1.0])
    
    # Project the 3D point onto the 2D image plane using the intrinsic matrix
    point_2d_homogeneous = K @ point_3d[:3]  # Matrix multiplication

    # Normalize to get (u, v) image coordinates
    u = point_2d_homogeneous[0] / point_2d_homogeneous[2]
    v = point_2d_homogeneous[1] / point_2d_homogeneous[2]

    return int(u), int(v)

In [5]:
def project_2d_to_3d(point_2d, K, depth):
    """
    Project a 2D point to 3D given the camera intrinsic matrix and the depth.

    :param point_2d: A tuple or list (u, v) representing the 2D image coordinates.
    :param K: Camera intrinsic matrix (3x3).
    :param depth: The depth (z-value) at the 2D point.
    :return: A numpy array [x, y, z] representing the 3D point in camera coordinates.
    """
    u, v = point_2d

    # Create the 2D point in homogeneous coordinates
    point_2d_homogeneous = np.array([u, v, 1.0])

    # Invert the intrinsic matrix
    K_inv = np.linalg.inv(K)

    # Compute the 3D point in camera coordinates
    point_3d = depth * K_inv @ point_2d_homogeneous

    return point_3d

In [2]:
def rpy_to_rotation_matrix(roll, pitch, yaw):
    """
    Convert Roll, Pitch, Yaw angles to a rotation matrix.
    
    Args:
        roll (float): Roll angle in radians.
        pitch (float): Pitch angle in radians.
        yaw (float): Yaw angle in radians.
        
    Returns:
        np.ndarray: 3x3 rotation matrix.
    """
    # Compute individual rotation matrices
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    
    R_y = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    
    R_z = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    
    # Combined rotation matrix: R = Rz * Ry * Rx
    R = R_z @ R_y @ R_x
    return R

In [4]:
def preprocess_pointcloud(pc1, nb_neighbors=50, std_ratio=0.5, voxel_size=0.02):
    pcd = copy.deepcopy(pc1)

    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    pcd = pcd.select_by_index(ind)
    
    voxel_size = voxel_size  # Adjust this parameter as needed
    pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

    pcd, centroid = center_point_cloud(pcd)

    return pcd, centroid

In [2]:
def create_pcd_from_rgbd(rgb_image, depth_image, intrinsics):
    # Convert numpy arrays to Open3D images
    o3d_rgb = o3d.geometry.Image(rgb_image.astype(np.uint8))
    o3d_depth = o3d.geometry.Image(depth_image.astype(np.float32))

    # Create an RGBDImage from the RGB and depth images
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color=o3d_rgb,
        depth=o3d_depth,
        depth_scale=1.0,    # Adjust if depth is not in meters (e.g., if in mm, use 1000.0)
        depth_trunc=1000.0, # Maximum depth cut off
        convert_rgb_to_intensity=False
    )

    # Create point cloud from RGBD image and intrinsics
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsics)

    return pcd

# Visualization

In [8]:
def visualize_points(img, points, original_size, resized_size, size=10):
    base_image = copy.deepcopy(img)
    draw = ImageDraw.Draw(base_image)
    
    for point in points:
        upsampled_point = upsample_2d_point(point, original_size, resized_size)
        draw.ellipse((upsampled_point[0] - size, upsampled_point[1] - size, upsampled_point[0] + size, upsampled_point[1] + size), fill="red")
        
    return base_image

In [1]:
def visualize_points_on_pointcloud(pc, pc_centroid, points, scaling_factor=1, radius=0.05):
    # Step1: Deep copy the input pointcloud
    A = copy.deepcopy(pc)
    
    # Step 2: Define the new points and set their color to red
    new_points = np.array(points) * scaling_factor
    new_points -= (pc_centroid * scaling_factor)
    new_colors = np.array([[1, 0, 0] for point in points])
    
    # Step 3: Combine the new points with the existing point cloud
    combined_points = np.vstack((np.asarray(A.points), new_points))
    combined_colors = np.vstack((np.asarray(A.colors), new_colors))
    
    # Update the original point cloud with combined data
    A.points = o3d.utility.Vector3dVector(combined_points)
    A.colors = o3d.utility.Vector3dVector(combined_colors)
    
    # Step 4: Create spheres for the new points to simulate increased size
    spheres = []
    for point in new_points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)  # Adjust the radius for desired size
        sphere.translate(point)  # Move sphere to the point location
        sphere.paint_uniform_color([1, 0, 0])  # Set color to red
        spheres.append(sphere)
    
    # Combine the point cloud and spheres for visualization
    geometry_list1 = [A] + spheres
    
    # Step 5: Visualize the updated point cloud with enlarged new points
    o3d.visualization.draw_geometries(geometry_list1)

In [1]:
def axis_angle_to_rotation_matrix(axis_angle):
    """
    Converts an axis-angle representation to a 3x3 rotation matrix.
    """
    rotation_matrix = R.from_rotvec(axis_angle).as_matrix()
    return rotation_matrix

def create_frame(position, rotation_matrix, size=0.2):
    """
    Creates a coordinate frame at a specific position and orientation.
    """
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    # Apply rotation
    frame.rotate(rotation_matrix, center=(0, 0, 0))
    # Apply translation
    frame.translate(position)
    return frame

def get_joint_frames(joint_positions, joint_angles):
    """
    Visualizes joint frames with the correct rotations applied.

    Args:
        joint_positions (np.ndarray): Nx3 array of joint positions.
        joint_angles (np.ndarray): Nx3 array of joint axis-angle rotations.
    """
    vis_objects = []
    for position, axis_angle in zip(joint_positions, joint_angles):
        # Convert axis-angle to rotation matrix
        rotation_matrix = axis_angle_to_rotation_matrix(axis_angle)
        # Create a coordinate frame for the joint
        frame = create_frame(position, rotation_matrix)
        vis_objects.append(frame)

    return vis_objects

# Math

In [5]:
def compute_xyz_distance(point1, point2):
    """
    Compute the Euclidean distance between two 3D points.
    
    Parameters:
    - point1: array-like, coordinates of the first point (x1, y1, z1).
    - point2: array-like, coordinates of the second point (x2, y2, z2).
    
    Returns:
    - distance: float, the Euclidean distance between the points.
    """
    # Convert inputs to numpy arrays for convenience
    point1 = np.array(point1)
    point2 = np.array(point2)
    
    # Compute the Euclidean distance
    distance = np.linalg.norm(point1 - point2)
    
    return distance

In [9]:
def compute_xy_distance(point1, point2):
    """
    Compute the Euclidean distance between two 3D points in the xy-plane.
    
    Parameters:
    - point1: array-like, coordinates of the first point (x1, y1, z1).
    - point2: array-like, coordinates of the second point (x2, y2, z2).
    
    Returns:
    - distance_xy: float, the Euclidean distance in the xy-plane.
    """
    # Extract x and y coordinates
    x1, y1 = point1[0], point1[1]
    x2, y2 = point2[0], point2[1]
    
    # Compute the 2D Euclidean distance
    distance_xy = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    
    return distance_xy

In [7]:
def angle_between_vectors(v1, v2):
    """
    Compute the angle (in radians) between two vectors.
    
    Parameters:
    - v1: array-like, the first vector
    - v2: array-like, the second vector
    
    Returns:
    - angle: float, the angle in radians between the two vectors
    """
    # Convert inputs to numpy arrays
    v1 = np.array(v1)
    v2 = np.array(v2)
    
    # Compute the dot product and magnitudes
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    
    # Prevent division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        raise ValueError("One of the vectors has zero magnitude.")
    
    # Compute the cosine of the angle
    cos_theta = dot_product / (magnitude_v1 * magnitude_v2)
    
    # Clip values to handle numerical issues (e.g., slight overflow beyond [-1, 1])
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    
    # Compute the angle in radians
    angle = np.arccos(cos_theta)
    
    return angle

In [11]:
def get_top_n_smallest_indices(values, n):
    """
    Return the indices of the top N smallest values from a list.
    
    Parameters:
    - values: list of numbers
    - n: int, number of smallest values to return
    
    Returns:
    - list: Indices of the top N smallest values in ascending order of value.
    """
    # Use heapq.nsmallest with indices
    return [idx for value, idx in heapq.nsmallest(n, [(v, i) for i, v in enumerate(values)])]

In [3]:
def chamfer_distance(pcd1, pcd2, R, t) -> float:
    """
    Compute the Chamfer Distance between two Open3D PointClouds.

    Parameters:
        pcd1 (o3d.geometry.PointCloud): First point cloud.
        pcd2 (o3d.geometry.PointCloud): Second point cloud.

    Returns:
        float: Chamfer distance between the two point clouds.
    """
    # Extract points from the point clouds as numpy arrays
    points1 = np.asarray(pcd1.points)
    points2 = np.asarray(pcd2.points)

    # Transform point1
    transformed_points1 = (R @ points1.T).T + t

    # Build KD-Trees for efficient nearest neighbor search
    tree1 = cKDTree(transformed_points1)
    tree2 = cKDTree(points2)

    # Compute the nearest neighbor distances from points1 to points2
    dist1, _ = tree1.query(points2)
    dist2, _ = tree2.query(points1)

    # Chamfer distance: mean of the nearest neighbor distances
    chamfer_dist = np.mean(dist1**2) + np.mean(dist2**2)

    return chamfer_dist

In [1]:
def compute_top_matches(cos_map, n_points, min_distance=50):
    """
    Select top points from a cosine similarity map with a constraint that they are at least
    a minimum distance apart based on a distance metric.
    
    Args:
        cos_map (np.ndarray): Cosine similarity map of shape (1, height, width).
        min_distance (float): Minimum distance between selected points.

    Returns:
        list: List of selected points (x, y) that satisfy the constraints.
    """
    # Parameters
    select = n_points
    top_n = 5000
    
    # Step 1: Flatten the map and get indices of top points
    flat_cos_map = cos_map[0].ravel()
    top_indices = np.argpartition(flat_cos_map, -top_n * 10)[-top_n * 10:]
    top_indices_sorted = top_indices[np.argsort(flat_cos_map[top_indices])[::-1]]
    
    # Step 2: Convert indices back to 2D coordinates
    top_xy_candidates = np.array(np.unravel_index(top_indices_sorted, cos_map[0].shape)).T[:, ::-1]
    
    # Step 3: Select points based on distance constraint
    selected_ref_points = []
    
    for candidate in top_xy_candidates:
        if len(selected_ref_points) == 0:
            selected_ref_points.append(candidate)
        else:
            # Calculate distances from the candidate to all previously selected points
            distances = cdist([candidate], selected_ref_points)
            if np.all(distances >= min_distance):
                # Optional: Check if candidate satisfies other conditions (e.g., masks_target)
                selected_ref_points.append(candidate)
            
            # Stop once we've collected enough points
            if len(selected_ref_points) >= select:
                break
    
    return selected_ref_points

In [1]:
def generate_random_rotation_matrix():
    """
    Generate a random 3x3 rotation matrix.
    A rotation matrix is an orthogonal matrix with determinant 1.

    Returns:
        numpy.ndarray: A 3x3 rotation matrix.
    """
    # Generate a random 3x3 matrix
    random_matrix = np.random.randn(3, 3)
    
    # Perform QR decomposition
    q, r = np.linalg.qr(random_matrix)
    
    # Ensure the determinant is 1 for a proper rotation matrix
    if np.linalg.det(q) < 0:
        q[:, 2] = -q[:, 2]
    
    return q

In [2]:
def apply_rotation_to_pointcloud(point_cloud, rotation_matrix):
    """
    Applies a rotation matrix to an Open3D point cloud.

    Args:
        point_cloud (o3d.geometry.PointCloud): The input point cloud.
        rotation_matrix (numpy.ndarray): The 3x3 rotation matrix to apply.

    Returns:
        o3d.geometry.PointCloud: The rotated point cloud.
    """
    # Get the numpy array of points from the point cloud
    points = np.asarray(point_cloud.points)
    
    # Apply the rotation matrix to the points
    rotated_points = np.dot(points, rotation_matrix)
    
    # Create a new point cloud with rotated points
    rotated_point_cloud = o3d.geometry.PointCloud()
    rotated_point_cloud.points = o3d.utility.Vector3dVector(rotated_points)
    
    # Preserve colors if present
    if point_cloud.has_colors():
        rotated_point_cloud.colors = point_cloud.colors
    
    return rotated_point_cloud

# OpenCV

In [1]:
def compute_corners(image, mask, max_corners=1000, quality_level=0.01, min_distance=10, plot=False):
    """
    Computes corners in the image within a specified mask using Shi-Tomasi Corner Detector.

    Args:
        image (np.ndarray): Input image.
        mask (np.ndarray): Binary mask where the corners are to be detected.
        max_corners (int): Maximum number of corners to detect.
        quality_level (float): Minimum accepted quality of corners.
        min_distance (float): Minimum possible Euclidean distance between the returned corners.
        plot (bool): Whether to plot the image with detected corners.

    Returns:
        np.ndarray: Detected corners within the masked region.
    """
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect corners using Shi-Tomasi Corner Detector
    corners = cv2.goodFeaturesToTrack(gray_image, max_corners, quality_level, min_distance, mask=mask)
    
    if corners is not None:
        corners = np.int0(corners)  # Convert to integer for pixel indexing
    
        # Filter corners by the mask
        filtered_corners = []
        for corner in corners:
            x, y = corner.ravel()
            if mask[y, x]:  # Check if the corner is within the masked region
                filtered_corners.append((x, y))
        
    #     filtered_corners = np.array(filtered_corners)
    # else:
    #     filtered_corners = np.array([])  # No corners found

    if plot:
        # Draw circles on the detected corners within the mask
        image_with_corners = image.copy()
        for corner in filtered_corners:
            x, y = corner
            cv2.circle(image_with_corners, (x, y), 5, (0, 255, 0), -1)  # Draw green circles
        
        # Display the original image with corners
        plt.figure(figsize=(10, 10))
        plt.title("Detected Corners in Masked Region")
        plt.imshow(cv2.cvtColor(image_with_corners, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display
        plt.axis('off')
        plt.show()

    return filtered_corners